In [52]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src/')

from math import ceil, floor, pi, acos, prod, sqrt, sin, cos, tan
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import cv2

from IPython.display import clear_output
import matplotlib
matplotlib.use('WebAgg')

from colorsys import hls_to_rgb
def colorize(z):
    n,m = z.shape
    c = np.zeros((n,m,3))
    c[np.isinf(z)] = (1.0, 1.0, 1.0)
    c[np.isnan(z)] = (0.5, 0.5, 0.5)

    idx = ~(np.isinf(z) + np.isnan(z))
    A = (np.angle(z[idx]) + np.pi) / (2*np.pi)
    A = (A + 0.5) % 1.0
    B = 1.0 - 1.0/(1.0+abs(z[idx])**0.3)
    c[idx] = [hls_to_rgb(a, b, 0.8) for a,b in zip(A,B)]
    return c

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
sys.path.append(os.getenv("HEDS_PYTHON_MODULES", ""))
from holoeye import slmdisplaysdk

ErrorCode = slmdisplaysdk.SLMDisplay.ErrorCode
ShowFlags = slmdisplaysdk.SLMDisplay.ShowFlags

slm = slmdisplaysdk.SLMDisplay()


ModuleNotFoundError: No module named 'holoeye'

In [ ]:
plt.close('all')
plt.draw_all()

In [ ]:
def on_press(event):
    print('press', event.key)
    # sys.stdout.flush()
    clear_output(wait=True)
    if event.key == 'x':
        visible = xl.get_visible()
        xl.set_visible(not visible)
        fig.canvas.draw()
    if event.key == 'z':
        plt.close()
        
def on_click(event):
    print('click', event.button, event.xdata, event.ydata)
    # sys.stdout.flush()
    clear_output(wait=True)

# Fixing random state for reproducibility
np.random.seed(19680801)

fig, ax = plt.subplots()

plt.connect('key_press_event', on_press)
plt.connect('button_press_event', on_click)

ax.plot(np.random.rand(12), np.random.rand(12), 'go')
xl = ax.set_xlabel('easy come, easy go')
ax.set_title('Press a key')
plt.show()

press z


In [102]:
slm_res = 8
slm_dim = np.array((1080, 1920))
lens_size = 64
wl = 0.810
deflect_angle = 0.5
offset = np.array([0, 0])

pattern_size, redun_size = np.divmod(slm_dim, lens_size)
k0 = 2*pi/wl

In [105]:
def slm_phaseamp_control(profile) -> np.ndarray:
    phase_plus = np.angle(profile) + np.arccos(np.abs(profile))
    phase_minus = np.angle(profile) - np.arccos(np.abs(profile))
    meta = np.array([[np.exp(1j*phase_plus), np.exp(1j*phase_minus)],
                     [np.exp(1j*phase_minus), np.exp(1j*phase_plus)]])
    meta = np.transpose(meta, (2, 0, 3, 1))
    meta = np.repeat(meta, lens_size//2, axis=0)
    meta = np.repeat(meta, lens_size//2, axis=2)
    meta = np.reshape(
        meta, (meta.shape[0]*meta.shape[1], meta.shape[2]*meta.shape[3]))
    return meta


def create_phase_gradient() -> np.ndarray:
    x = np.arange(0, slm_dim[0]*slm_res, slm_res)
    y = np.arange(0, slm_dim[1]*slm_res, slm_res)
    y, x = np.meshgrid(y, x)
    pg = np.exp(1j*k0*sin(deflect_angle)*x)
    return pg


def slm_gradient_control(profile) -> np.ndarray:
    constant = np.ones(
        (pattern_size[0]*lens_size, pattern_size[1]*lens_size), dtype=complex)
    gradient = phase_gradient[:pattern_size[0]*lens_size, :pattern_size[1]*lens_size]
    cond = np.kron(profile, np.ones((lens_size, lens_size)))
    meta = np.where(cond, gradient, constant)
    return meta


phase_gradient = create_phase_gradient()

profile = np.zeros(pattern_size, dtype=complex)
pattern = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

profile[offset[0]:offset[0]+pattern.shape[0],
        offset[1]:offset[1]+pattern.shape[1]] = pattern

slm_phase = np.angle(slm_phaseamp_control(profile) *
                     slm_gradient_control(np.abs(profile) == 0))
slm_phase = np.pad(
    slm_phase, ((0, redun_size[0]), (0, redun_size[1])), 'constant')


In [111]:
import matplotlib.pyplot as plt
import numpy as np

from mpl_interactions import ipyplot as iplt

plt.close('all')

lens_size = 64
offset = np.array([0, 0])
deflect_angle = 0.5

def img(offset0, offset1, lens_size0, deflect_angle0):
    global lens_size, pattern_size, redun_size, deflect_angle, phase_gradient
    try:
        offset[0] = offset0
        offset[1] = offset1
        lens_size = lens_size0
        pattern_size, redun_size = np.divmod(slm_dim, lens_size)
        if deflect_angle != deflect_angle0:
            deflect_angle = deflect_angle0
            phase_gradient = create_phase_gradient()

        profile = np.zeros(pattern_size, dtype=complex)
        profile[offset[0]:offset[0]+pattern.shape[0],
                offset[1]:offset[1]+pattern.shape[1]] = pattern

        slm_phase = np.angle(slm_phaseamp_control(
            profile)*slm_gradient_control(np.abs(profile) == 0))
        slm_phase = np.pad(
            slm_phase, ((0, redun_size[0]), (0, redun_size[1])), 'constant')
    except Exception as e:
        slm_phase = np.zeros(slm_dim)
        # raise (e)
    return slm_phase


fig, ax = plt.subplots()
controls = iplt.imshow(img, 
    offset0=range(0, pattern_size[0]), 
    offset1=range(0, pattern_size[1]), 
    lens_size0=range(lens_size//2, 3*lens_size//2, 2), 
    deflect_angle0=(deflect_angle/2, deflect_angle*3/2), 
    cmap='hsv')
controls.controls['lens_size0'].set_val(lens_size//2)
plt.show()
